In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
# import plotly as px
from pyproj import Transformer
import requests

In [14]:
weer_api = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\apidata_2021.csv")
weer= pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\weerstations_noordholland.csv", sep= ";")
weer = gpd.GeoDataFrame(weer, geometry=gpd.GeoSeries.from_wkt(weer['WKT']))
weer['STN']=weer['STN_y']
weer.drop(columns=['WKT', 'STN_x', 'STN_y'], inplace=True)
hectointervallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\hectointervallen.txt") 
hectointervallen = hectointervallen[hectointervallen.nunique()[hectointervallen.nunique() != 1].index]
hectopunten = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\hectopunten.txt") 
hectopunten = hectopunten[hectopunten.nunique()[hectopunten.nunique() != 1].index]
juncties = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\juncties.txt") 
juncties = juncties[juncties.nunique()[juncties.nunique() != 1].index]
puntlocaties = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Netwerkgegevens\puntlocaties.txt") 
puntlocaties = puntlocaties[puntlocaties.nunique()[puntlocaties.nunique() != 1].index]
ongevallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Ongevallengegevens\ongevallen.txt", encoding="ISO-8859-15")
ongevallen = ongevallen[ongevallen.nunique()[ongevallen.nunique() != 1].index]
ongevallen.drop(columns=['PVOPGEM', 'WVL_ID','WVG_ID', 'WVG_AN', 'WDK_ID', 'WDK_AN',
                         'LGD_ID', 'BZD_ID_VM1','BZD_ID_VM2', 'BZD_ID_VM3', 'BZD_VM_AN', 'BZD_ID_IF1', 'BZD_ID_IF2',
                         'BZD_ID_IF3', 'BZD_IF_AN', 'BZD_ID_TA1', 'BZD_ID_TA2', 'BZD_ID_TA3', 'HUISNUMMER'], inplace=True)


C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3399033372.py:14: DtypeWarning: Columns (1,24,39) have mixed types. Specify dtype option on import or set low_memory=False.
  ongevallen = pd.read_csv(r"C:\Users\tjibb\Documents\school\jaar_3\minor\eind_prestentatie\verkeer\PGS0112-o-CSV-bestand-J-1-N-N-N\Ongevallengegevens\ongevallen.txt", encoding="ISO-8859-15")


In [3]:
samengevoegd_df = ongevallen.merge(juncties, on='JTE_ID', how='left')
samengevoegd_df['FK_VELD5']=samengevoegd_df['FK_VELD5_y']
samengevoegd_df = samengevoegd_df.merge(puntlocaties, on='FK_VELD5', how='left')
samengevoegd_df = samengevoegd_df.merge(puntlocaties, on='FK_VELD5', how='left')
df= samengevoegd_df
samengevoegd_df = df[df.nunique()[df.nunique() != 1].index]
samengevoegd_df.drop(columns=['AP4_CODE', 'AP5_CODE', 'FK_VELD5_y', 'X_COORD_y', 'Y_COORD_y', 
                              'ANTL_SLA', 'ANTL_DOD', 'ANTL_GZH', 'DATUM_VKL', 'DAG_CODE', 'MND_NUMMER',
                              'TIJDSTIP', 'UUR', 'DDL_ID', 'DAGTYPE', 'WEEKNR',
                              'ANTL_SEH','ANTL_GOV','ANTL_TDT', 'MNE_CODE', 'ZAD_ID', 'WGD_CODE_2',
                              'IND_ALC'], inplace=True)

df= samengevoegd_df
# df.columns
df2=df[['X_COORD_x', 'Y_COORD_x', 'VKL_NUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID', 'BEBKOM', 'WGD_CODE_1']]
df2['X']=df2['X_COORD_x']
df2['Y']=df2['Y_COORD_x']
df2.drop(columns=['X_COORD_x', 'Y_COORD_x'], inplace=True)

C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3343418178.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['X']=df2['X_COORD_x']
C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3343418178.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Y']=df2['Y_COORD_x']
C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3343418178.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
samengevoegd_df.columns

Index(['VKL_NUMMER', 'REGNUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID',
       'NIVEAUKOP', 'WSE_ID', 'BEBKOM', 'MAXSNELHD', 'WGD_CODE_1', 'JTE_ID',
       'WVK_ID', 'HECTOMETER', 'FK_VELD5_x', 'GME_ID', 'GME_NAAM', 'PVE_CODE',
       'PVE_NAAM', 'KDD_NAAM', 'PLT_NAAM', 'DIENSTCODE', 'DIENSTNAAM',
       'DISTRCODE', 'DISTRNAAM', 'WBRSRT_R', 'WBRSRT_P', 'WBRSRT_G',
       'WBRSRT_W', 'WBRSRT_T', 'ANTL_TAK', 'ACTUEEL', 'SLE_TYPE', 'SLE_NUMMER',
       'FK_VELD5', 'X_COORD_x', 'Y_COORD_x'],
      dtype='object')

In [5]:
trans = Transformer.from_crs("ETRS89", "EPSG:4326", always_xy=True)
xx, yy = trans.transform(df2["X"].values, df2["Y"].values)

df2["X"] = xx
df2["Y"] = yy
df2["X"] = (df2["X"]/64000)+3
df2["Y"] = (df2["Y"]/110000)+48
# df2.drop(columns=['X_COORD_x', 'Y_COORD_x'], inplace=True)

df2=df2[['X', 'Y', 'VKL_NUMMER', 'AP3_CODE', 'ANTL_PTJ', 'AOL_ID', 'BEBKOM', 'WGD_CODE_1']]
df2.dropna()
gdf = gpd.GeoDataFrame(
    df2, geometry=gpd.points_from_xy(df2.X, df2.Y))
gdf.dropna()

C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3967659008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["X"] = xx
C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3967659008.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Y"] = yy
C:\Users\tjibb\AppData\Local\Temp\ipykernel_12476\3967659008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,X,Y,VKL_NUMMER,AP3_CODE,ANTL_PTJ,AOL_ID,BEBKOM,WGD_CODE_1,geometry
8,5.698422,51.700464,20210000435,DOD,2,7,BI,D,POINT (5.69842 51.70046)
16,5.440999,51.761055,20210000567,DOD,2,7,BU,D,POINT (5.44100 51.76105)
17,6.486673,52.904554,20210000568,DOD,2,7,BU,D,POINT (6.48667 52.90455)
25,4.427013,51.588056,20210072804,UMS,2,7,BI,D,POINT (4.42701 51.58806)
29,6.990462,53.006926,20210074980,UMS,2,7,BI,D,POINT (6.99046 53.00693)
...,...,...,...,...,...,...,...,...,...
113144,5.783843,51.445289,20210027300,UMS,3,7,BU,D,POINT (5.78384 51.44529)
113147,6.821636,53.241292,20210027330,UMS,2,0,BI,D,POINT (6.82164 53.24129)
113150,4.806394,52.453405,20210027591,UMS,3,9,BI,D,POINT (4.80639 52.45340)
113151,4.558489,52.098595,20210027600,UMS,3,7,BI,D,POINT (4.55849 52.09860)


In [6]:
weer_api

,Unnamed: 0,datetime,Windsnelheid (km/h),Windrichting (Graden),Luchtdruk (ps),Zichtbaarheid (Decimeter ver kunnen kijken),Regenval (mm/h),station_id,Temperatuur (C),Weercode
0,0,2021-01-01 00:00:00+00:00,0.80,232.2,1005.84,123.0,0.0,6260,-2.1,0.0
1,1,2021-01-01 00:10:00+00:00,0.90,233.2,1005.84,126.0,0.0,6260,-2.0,0.0
2,2,2021-01-01 00:20:00+00:00,0.57,233.5,1005.94,109.0,0.0,6260,-2.2,0.0
3,3,2021-01-01 00:30:00+00:00,0.39,233.1,1005.94,137.0,0.0,6260,-2.3,0.0
4,4,2021-01-01 00:40:00+00:00,0.59,232.1,1005.94,142.0,0.0,6260,-2.3,0.0
...,...,...,...,...,...,...,...,...,...,...
559279,559279,2021-12-30 23:20:00+00:00,13.65,231.3,1012.47,8510.0,0.0,6242,10.5,0.0
559280,559280,2021-12-30 23:30:00+00:00,13.24,230.8,1012.57,8590.0,0.0,6242,10.5,0.0
559281,559281,2021-12-30 23:40:00+00:00,13.11,231.4,1012.37,8720.0,0.0,6242,10.6,0.0
559282,559282,2021-12-30 23:50:00+00:00,13.20,230.2,1012.37,8880.0,0.0,6242,10.5,0.0


In [16]:
weer = pd.merge(weer, weer_api, on='station_id')
weer.drop(columns=['fid', 'Unnamed: 0', 'datetime', 'NAME'], inplace=True)


In [22]:
weer

,LON(east),LAT(north),ALT(m),geometry,station_id,STN,Windsnelheid (km/h),Windrichting (Graden),Luchtdruk (ps),Zichtbaarheid (Decimeter ver kunnen kijken),Regenval (mm/h),Temperatuur (C),Weercode
0,5.180,52.100,1.9,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1...",6260,260,0.80,232.2,1005.84,123.0,0.0,-2.1,0.0
1,5.180,52.100,1.9,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1...",6260,260,0.90,233.2,1005.84,126.0,0.0,-2.0,0.0
2,5.180,52.100,1.9,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1...",6260,260,0.57,233.5,1005.94,109.0,0.0,-2.2,0.0
3,5.180,52.100,1.9,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1...",6260,260,0.39,233.1,1005.94,137.0,0.0,-2.3,0.0
4,5.180,52.100,1.9,"MULTIPOLYGON (((5.04700 52.16696, 5.04699 52.1...",6260,260,0.59,232.1,1005.94,142.0,0.0,-2.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
559279,4.921,53.241,10.8,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1...",6242,242,13.65,231.3,1012.47,8510.0,0.0,10.5,0.0
559280,4.921,53.241,10.8,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1...",6242,242,13.24,230.8,1012.57,8590.0,0.0,10.5,0.0
559281,4.921,53.241,10.8,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1...",6242,242,13.11,231.4,1012.37,8720.0,0.0,10.6,0.0
559282,4.921,53.241,10.8,"MULTIPOLYGON (((4.75721 53.12708, 4.75766 53.1...",6242,242,13.20,230.2,1012.37,8880.0,0.0,10.5,0.0


In [23]:
weer_omtemergen= weer[['geometry', 'STN']]

In [25]:
weer_omtemergen.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 559284 entries, 0 to 559283
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   geometry  559284 non-null  geometry
 1   STN       559284 non-null  int64   
dtypes: geometry(1), int64(1)
memory usage: 12.8 MB


In [17]:
gdf = gdf.dropna()

In [18]:
weer.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 559284 entries, 0 to 559283
Data columns (total 13 columns):
 #   Column                                       Non-Null Count   Dtype   
---  ------                                       --------------   -----   
 0   LON(east)                                    559284 non-null  float64 
 1   LAT(north)                                   559284 non-null  float64 
 2   ALT(m)                                       559284 non-null  float64 
 3   geometry                                     559284 non-null  geometry
 4   station_id                                   559284 non-null  int64   
 5   STN                                          559284 non-null  int64   
 6   Windsnelheid (km/h)                          559284 non-null  float64 
 7   Windrichting (Graden)                        559284 non-null  float64 
 8   Luchtdruk (ps)                               559284 non-null  float64 
 9   Zichtbaarheid (Decimeter ver kunnen kijk

In [26]:
merged_gdf = gpd.sjoin(gdf, weer_omtemergen, predicate='within')


: 